# DeepLabCut Toolbox - Open-Field DEMO

Some resources that can be useful:

- [github.com/DeepLabCut/DeepLabCut](https://github.com/DeepLabCut/DeepLabCut)
- [DeepLabCut's Documentation: User Guide for Single Animal projects](https://deeplabcut.github.io/DeepLabCut/docs/standardDeepLabCut_UserGuide.html)

#### The notebook accompanies the following user-guide:

Nath\*, Mathis\* et al. *Using DeepLabCut for markerless pose estimation during behavior across species* Nature Protocols, 2019: https://www.nature.com/articles/s41596-019-0176-0

This notebook illustrates how to:
- load the demo project
- train a network
- evaluate a network
- analyze a novel video
- create an automatically labeled video 
- plot the trajectories 
- identify outlier frames
- annotate the outlier frames manually
- merge the data sets and update the training set
- train a network

Note: This notebook starts from an already initialized project with labeled data.


The data is a subset from *DeepLabCut: markerless pose estimation of user-defined body parts with deep learning* https://www.nature.com/articles/s41593-018-0209-y (this subset was not used to train models that are shown or evaluated in our paper).

In [ ]:
# Importing the toolbox (takes several seconds)
from pathlib import Path

import deeplabcut

In [ ]:
# Create a variable to set the config.yaml file path:
# If this path does not point to the project from the URL below,
# edit it to make sure it does:
#   https://github.com/DeepLabCut/DeepLabCut/tree/main/examples/openfield-Pranav-2018-10-30
# 
# Example - Linux/OSX
#   path_config_file = "/Users/john/DeepLabCut/examples/openfield-Pranav-2018-10-30/config.yaml"
# Example - Windows
#   path_config_file = r"C:\DeepLabCut\examples\openfield-Pranav-2018-10-30\config.yaml"
#
# Note that parameters of this project can be seen at: *openfield-Pranav-2018-10-30/config.yaml*

path_config_file = str(Path.cwd() / "openfield-Pranav-2018-10-30" / "config.yaml")
deeplabcut.load_demo_data(path_config_file)

In [ ]:
# [OPTIONAL] Perhaps plot the labels to see how the frames were annotated:
# (note, this project was created in Linux, so you might have an error in Windows, but this is an optional step)

deeplabcut.check_labels(path_config_file)

## Start training of Feature Detectors

This function trains the network for a specific shuffle of the training dataset. The user can set various parameters in `/openfield-Pranav-2018-10-30/dlc-models-pytorch/.../pytorch_config.yaml`. For more information about the variables that can be set, check out the [docs](https://deeplabcut.github.io/DeepLabCut/docs/pytorch/pytorch_config.html)!

Training can be stopped at any time. Note that the weights are only stored every 'save_epochs' epochs. For this demo the state it is advisable to store & display the progress very often. In practice this is inefficient. You should see the model start converging around 50 to 60 epochs; you can continue training it longer to improve performance.

In [ ]:
# notice the variables "save_epochs" and "displayiters" that can be set in the function
deeplabcut.train_network(
    path_config_file,
    shuffle=1,
    save_epochs=2,
    displayiters=5,
)

**Note, that if it reaches the end or you stop it (by hitting "stop" or by CTRL+C), 
you will see an "KeyboardInterrupt" error, but you can ignore this!**

## Evaluate a trained network

This function evaluates a trained model for a specific shuffle/shuffles at a particular training state (snapshot) or on all the states. The network is evaluated on the data set (images) and stores the results as .csv file in a subdirectory under **evaluation-results-pytorch**.

You can change various parameters in the ```config.yaml``` file of this project. For evaluation all the model descriptors (Task, TrainingFraction, Date etc.) are important. For the evaluation one can change pcutoff. This cutoff also influences how likely estimated positions need to be so that they are shown in the plots. One can furthermore, change the colormap and dotsize for those graphs.

In [ ]:
deeplabcut.evaluate_network(path_config_file, plotting=False)

*NOTE: depending on your set up sometimes you get some "matplotlib errors, but these are not important*

Now you can go check out the images. Given the limited data input and it took ~20 mins to test this out, it is not meant to track well, so don't be alarmed. This is just to get you familiar with the workflow... 

## Analyzing videos
This function extracts the pose based on a trained network from videos. The user can choose the trained network - by default the most recent snapshot is used to analyse the videos. However, the user can also specify the snapshot index for the variable **snapshotindex** in the **config.yaml** file).

The results are stored in hd5 file in the same directory, where the video resides. The pose array (pose vs. frame index) can also be exported as csv file (set flag to...). 

In [ ]:
videofile_path = str(Path(path_config_file).parent / "videos" / "m3v1mp4.mp4")

In [ ]:
print("Start analyzing the video!")
# our demo video on a CPU with take ~5 min to analze! GPU is much faster!
deeplabcut.analyze_videos(path_config_file, [videofile_path])

## Create labeled video

This function is for the visualization purpose and can be used to create a video in .mp4 format with the predicted labels. This video is saved in the same directory, where the (unlabeled) video resides. 

Various parameters can be set with regard to the colormap and the dotsize. The parameters of the 

In [ ]:
deeplabcut.create_labeled_video(path_config_file, [videofile_path])

## Plot the trajectories of the analyzed videos
This function plots the trajectories of all the body parts across the entire video. Each body part is identified by a unique color. The underlying functions can easily be customized.

In [ ]:
%matplotlib notebook
deeplabcut.plot_trajectories(
    path_config_file,
    [videofile_path],
    showfigures=True,
)

# These plots are interactive and can be customized (see https://matplotlib.org/)

## Extract outlier frames, where the predictions are off.

This is optional step allows to add more training data when the evaluation results are poor. In such a case, the user can use the following function to extract frames where the labels are incorrectly predicted. Make sure to provide the correct value of the "iterations" as it will be used to create the unique directory where the extracted frames will be saved.

In [ ]:
deeplabcut.extract_outlier_frames(path_config_file, [videofile_path])

The user can run this iteratively, and (even) extract additional frames from the same video.

## Manually correct labels

This step allows the user to correct the labels in the extracted frames. Navigate to the folder corresponding to the video 'm3v1mp4' and use the GUI as described in the protocol to update the labels.

For documentation regarding the GUI, [look at the docs for `napari-deeplabcut`](https://github.com/DeepLabCut/napari-deeplabcut/tree/main) - and specifically _"3. Refining labels – the image folder contains a machinelabels-iter<#>.h5 file."_!

In [ ]:
deeplabcut.refine_labels(path_config_file)

In [ ]:
# Perhaps plot the labels to see how how all the frames are annotated (including the refined ones)
deeplabcut.check_labels(path_config_file)

In [ ]:
# Now merge datasets (once you refined all frames)
deeplabcut.merge_datasets(path_config_file)

## Create a new iteration of training dataset, check it and train...

Following the refine labels, append these frames to the original dataset to create a new iteration of training dataset.

In [ ]:
deeplabcut.create_training_dataset(path_config_file)

Now one can train the network again... (with the expanded data set). We can continue training from the snapshot we already have by using the `snapshot_path` argument - instead of training the model from scratch, it will load the weights we already have and fine-tune them!

In [ ]:
snapshot_path = (  # Edit me if needed! Select the path to the snapshot to continue training from!
    Path(path_config_file).parent / 
    "dlc-models-pytorch" / 
    "iteration-0" / 
    "openfieldOct30-trainset95shuffle1" / 
    "train" / 
    "snapshot-best-080.pt"
)

deeplabcut.train_network(
    path_config_file,
    shuffle=1,
    save_epochs=2,
    displayiters=10,
    batch_size=8,
    snapshot_path=snapshot_path,
)